### 2021_10_14_5

## 대표적인 알고리즘 RandomForest

### 학습 내용
* 앙상블이 무엇인지 알아본다.
* 랜덤 포레스트 알고리즘을 이용해 본다.
* 기타 모델로 다시 만들어 여러 모델을 비교해 본다.

### 01 앙상블(Ensemble)이란?
* 여러 머신러닝 모델을 연결하여 더 강력한 모델을 만드는 기법이다.

### 02 RandomForest(랜덤 포레스트)는 무엇인가?
* 01 트리를 많이 만든다.
* 02 각각의 모델이 예측한다.
* 03 예측한 값들의 평균값을 구한다.
* 04 이를 최종 예측값으로 이용한다.

In [1]:
import seaborn as sns

In [2]:
tips = sns.load_dataset("tips")
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


In [3]:
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


* 데이터 셋 내용
	* total_bill : 총 지불 비용
	* tip : 팁
	* sex : 성별
	* smoker : 흡연 유무
	* day : 이용한 요일
	* time : 점심인지 저녁인지
	* size : 식당 이용 인원

### 04 머신러닝 문제
* 조건 1. 우리에게는 지금까지 이용한 고객의 180개의 데이터가 있다.
* 조건 2. 이후에 몇명이 이용할지 모른다.
* 조건 3. 우리는 몇명이 우리 식당에 방문할지 알아야 한다. 이 인원을 예측하는 머신러닝 시스템을 만들어, 이를 토대로 앞으로의 고객 서비스에 반영해보자.

In [5]:
tips_have = tips.iloc[0 : 220, :] # 현재 가진 고객 데이터
tips_new = tips.iloc[220 :, :]    # 미래의 고객 데이터

tips_new.drop(["size"], axis = 1, inplace = True)

tips_have.shape, tips_new.shape

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


((220, 7), (24, 6))

In [6]:
tips_have.columns, tips_new.columns

(Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object'),
 Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time'], dtype='object'))

### 05 머신러닝 문제 수행
* 머신러닝은 숫자 데이터를 좋아하고 이해할 수 있다.
  * 그러면 total_bill, tip의 컬럼(변수)를 사용해서 size를 예측하는 것을 해보자.

### 머신러닝은 다음과 같은 과정을 거친다.

* 모델 만들고
* 선택된 모델을 준비된 데이터(입력, 출력)로 학습을 시키고
* 마지막으로 학습된 모델로 새로운 데이터를 예측을 수행한다.

### 우리의 과제

* 모델에 사용할 데이터를 준비한다.
  * 학습-입력(X_train), 학습-출력(y_train)
  * 예측에 사용할 새로운 데이터(X_test), y_test(는 예측되므로 없음)

In [11]:
sel = ["total_bill", "tip"]

X = tips_have[sel]
y = tips_have["size"]

test_X = tips_new[sel]

### 랜덤 포레스트 이용

* 예측하려는 타깃(레이블)이 수치형일때는 RandomForestRegressor를 활용
* 예측하려는 타깃(레이블)이 범주형일때는 RandomForestClassifier를 활용

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
model = RandomForestClassifier().fit(X, y)
model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
pred = model.predict(test_X)
pred

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4,
       2, 4])

### 06 우리가 만든 모델이 좋은지 아닌지 어떻게 평가할 수 없을까?

* 내가 만든 모델이 어느정도 좋은 성능을 가지는지 현재로서는 알기가 어렵다.
  * 해결 방안 1. tips_have에는 출력 size가 있다. tips_new는 없다. 그러면 우선 tips_have을 잘 데이터로 나누어 학습과 예측을 하여, 가진 답으로 맞추어보고 검증을 해보자.
* train_test_split 함수를 이용하여 학습용, 테스트용으로 나눌 수 있다.

In [14]:
from sklearn.model_selection import train_test_split

In [18]:
# random_state는 난수 발생기의 패턴을 고정시키기 위해 사용한다.
# 이를 통해 우리는 X(입력), y(출력)이 각각 학습용, 테스트용으로 나누어진다.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

### 07 모델 생성 후, 평가

In [38]:
model = RandomForestClassifier(n_estimators = 100).fit(X_train, y_train)
pred = model.predict(X_test)
pred[0 : 5]

array([2, 2, 2, 2, 3])

### 여기서 예측한 pred와 y_test는 비교하여 얼마나 정확하게 맞추었는지 확인 가능

In [39]:
(pred == y_test).sum() / len(pred)

0.6

### 08 다른 모델의 정확도 확인

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [20]:
model = LinearRegression().fit(X_train, y_train)
pred = model.predict(X_test)

(pred == y_test).sum()/len(pred)

0.0

In [21]:
model = KNeighborsRegressor().fit(X_train, y_train)
pred = model.predict(X_test)

(pred == y_test).sum()/len(pred)

0.3090909090909091

In [49]:
model = DecisionTreeRegressor().fit(X_train, y_train)
pred = model.predict(X_test)

(pred == y_test).sum()/len(pred)

0.5272727272727272

### 결과 확인

* 일반적인 모델 사용 결과 knn보다 의사결정트리가 좋고,
* 의사결정트리보다 랜덤포레스트 모델이 좋다.
* 랜덤포레스트는 많은 개수의 트리를 사용해서, 많은 트리를 사용하는 것이 좋은 것으로 보여진다.